In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2
import random
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [2]:
data = pd.read_csv('data/Data_Entry_2017.csv')
bbox = pd.read_csv('data/BBox_List_2017.csv')
train_list = pd.read_csv('data/train_val_list.txt', sep="\n", header = None)
test_list = pd.read_csv('data/test_list.txt', sep="\n", header = None)

In [4]:
cm = []
for n in np.arange(len(data['Finding Labels'])):
    if ('Effusion' in data['Finding Labels'][n]):
        cm.append(1)
    else:
        cm.append(0)



data2 = data.copy()
data2['Effusion'] = cm
data2 = data2[['Image Index', 'Finding Labels', 'Effusion']]

In [7]:
data2.head(25)

,Image Index,Finding Labels,Effusion
0,00000001_000.png,Cardiomegaly,0
1,00000001_001.png,Cardiomegaly|Emphysema,0
2,00000001_002.png,Cardiomegaly|Effusion,1
3,00000002_000.png,No Finding,0
4,00000003_000.png,Hernia,0
5,00000003_001.png,Hernia,0
6,00000003_002.png,Hernia,0
7,00000003_003.png,Hernia|Infiltration,0
8,00000003_004.png,Hernia,0
9,00000003_005.png,Hernia,0


In [8]:
training = train_list[0].tolist()
training2 = random.sample(population = training, k = 1000)
d2 = data2[data2['Image Index'].isin(training2)]
d2

,Image Index,Finding Labels,Effusion
90,00000017_002.png,No Finding,0
248,00000054_006.png,Infiltration,0
360,00000091_000.png,No Finding,0
377,00000093_002.png,No Finding,0
495,00000118_001.png,Atelectasis|Emphysema,0
...,...,...,...
111771,00030621_006.png,Atelectasis|Consolidation|Infiltration,0
111940,00030682_001.png,No Finding,0
111942,00030683_001.png,Atelectasis,0
112001,00030722_000.png,Nodule,0


In [10]:
training_data = []
IMG_SIZE = 512
dim = (IMG_SIZE, IMG_SIZE)


def create_training():
    imagefiles = ['images_001','images_002','images_003','images_004','images_005',
                 'images_006','images_007','images_008','images_009','images_010',
                 'images_011','images_012']
    image_path = os.path.join('data','images')
    for file in os.listdir(image_path):
        if (file != '.DS_Store'):
            collection = os.path.join(image_path, file)
            imagefile = os.path.join(collection,'images')
            for pic in os.listdir(imagefile):
                if (pic in training2):
                    img_array = cv2.imread(os.path.join(imagefile, pic), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array, dim)
                    label = d2[d2['Image Index'].str.contains(pic)].reset_index()['Effusion'][0]
                    training_data.append([new_array, label])
                    

create_training()

In [11]:
print(len(training_data))

1000


In [12]:
random.shuffle(training_data)

In [15]:
X_eff = []
Y_eff = []

In [16]:
for features, label in training_data:
    X_eff.append(features)
    Y_eff.append(label)

X_eff = np.array(X_eff).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [17]:
pickle_out = open("X_eff.pickle","wb")
pickle.dump(X_eff, pickle_out)
pickle_out.close()

pickle_out = open("Y_eff.pickle","wb")
pickle.dump(Y_eff, pickle_out)
pickle_out.close()